In [1]:
import pandas as pd
from IPython.display import HTML
from tools.utils import *
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

## 标准答案处理

In [2]:
dev_df = pd.read_excel('dev.xlsx')
dev_df = pd.concat([dev_df,dev_df.label.apply(get_features,args=('A',))],axis=1)
disp(dev_df.head().drop(columns=['label']))

['api_1的结果', 'api_2的结果', {'api_3的结果 方向': '流入', 'api_4的结果 方向': '流出'}]


,query,A_api_list,A_api_params,A_api_num,A_result
0,我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')",[['国金核心资产一年持有期混合型证券投资基金A类']]['api_0的结果']['api_1的结果']['api_1的结果'],4,"api_1,api_2,api_3"
1,我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询规模')('基金查询', '查询申购费率')",[['汇丰晋信2016生命周期开放式证券投资基金C类']]['api_0的结果']['api_0的结果']['api_0的结果'],4,"api_1,api_2,api_3"
2,我现在要买1000份兴业聚惠A基金，我需要支付多少申购费？,"('基金查询', '查询代码')('基金查询', '查询申购费率')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')","[['兴业聚惠混合型证券投资基金A类']]['api_0的结果']['api_0的结果']['api_2的结果', '1000']['api_1的结果', 'api_3的结果']",5,api_4
3,南方新能源C的基金经理是谁，他管理的基金总规模有多大？,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')",[['南方新能源产业趋势混合型证券投资基金C类']]['api_0的结果']['api_1的结果'],3,"api_1,api_2"
4,汇添富添添鑫A基金近1年的最大回撤及其同类排名信息是怎样的？,"('基金查询', '查询代码')('基金查询', '查询近期最大回撤')('基金查询', '查询近期最大回撤同类排名')","[['汇添富添添鑫多元收益9个月持有期混合型证券投资基金A类']]['api_0的结果', '1年']['api_0的结果', '1年']",3,"api_1,api_2"


## 预测结果处理

In [3]:
def predict_process(predict_str:str):
    substr = '{"relevant APIs":'
    start_index = predict_str.find(substr)

    if start_index != -1:
        result = predict_str[start_index:]
    else:
        result = ""
    return result

predict = pd.read_table('generated_predictions.txt',header=None)#.apply(lambda s:eval(s)['label'])
predict_label = predict[0].apply(lambda s:eval(s)['predict'])
predict_label = predict_label.apply(predict_process)
predict_features = predict_label.apply(get_features,args=('B',))

error in parsing 
error in parsing {"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["中光防雷"]], "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "查询当前价", "required_parameters": ["api_0的结果"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "乘法计算", "required_parameters": ["api_1的结果", "10000"], "rely_apis": ["1"], "tool_name": "数值计算"}, {"api_id": "3", "api_name": "查询成交额", "required_parameters": ["api_0的结果"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "4", "api_name": "乘法计算", "required_parameters": ["api_2的结果", "api_3的结果"], "rely_apis": ["2", "3"], "tool_name": "数值计算"}, {"api_id": "5", "api_name": "减法计算", "required_parameters": ["api_2的结果", "api_4的结果"], "rely_apis": ["2", "4"], "tool_name": "数值计算"}, {"api_id": "6", "api_name": "减法计算", "required_parameters": ["api_2的结果", "api_4": ["api_5的结果"], "rely_apis": ["2", "5"], "tool_name": "数值计算"}], "result": ["api_6的结果"]}
error in parsing 
error in parsing 
error in pars

## 合并 & 比较

In [4]:
compare_df = pd.concat([dev_df.drop(columns=['label']),predict_features],axis=1)
compare_df = compare_df.reindex(columns=['query','A_api_list','B_api_list','A_api_params','B_api_params','A_api_num','B_api_num','A_result','B_result'])

In [5]:
compare_df[['A_api_list','B_api_list']] = compare_df.apply(highlight_differences, args=('A_api_list','B_api_list'), axis=1)
compare_df[['A_api_params','B_api_params']] = compare_df.apply(highlight_differences, args=('A_api_params','B_api_params'), axis=1)
disp(compare_df.head(),cols_width=[200,300,300,300,300,100,100,100,100])

,query,A_api_list,B_api_list,A_api_params,B_api_params,A_api_num,B_api_num,A_result,B_result
0,我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')",[['国金核心资产一年持有期混合型证券投资基金A类']]['api_0的结果']['api_1的结果']['api_1的结果'],[['国金核心资产一年持有期混合型证券投资基金A类']]['api_0的结果']['api_1的结果']['api_1的结果'],4,4,"api_1,api_2,api_3","api_2,api_3"
1,我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')",[['汇丰晋信2016生命周期开放式证券投资基金C类']]['api_0的结果']['api_0的结果'],[['汇丰晋信2016生命周期开放式证券投资基金C类']]['api_0的结果']['api_1的结果'],4,4,"api_1,api_2,api_3","api_2,api_3"
2,我现在要买1000份兴业聚惠A基金，我需要支付多少申购费？,"('基金查询', '查询代码')('基金查询', '查询申购费率')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')","[['兴业聚惠混合型证券投资基金A类']]['api_0的结果']['api_0的结果']['api_1的结果', 'api_3的结果']","[['兴业聚惠混合型证券投资基金A类']]['api_0的结果']['api_1的结果', '1000']",5,4,api_4,api_3
3,南方新能源C的基金经理是谁，他管理的基金总规模有多大？,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')",[['南方新能源产业趋势混合型证券投资基金C类']]['api_1的结果'],[['南方中证新能源交易型开放式指数证券投资基金联接基金C类']]['api_1的结果'],3,3,"api_1,api_2","api_1,api_2"
4,汇添富添添鑫A基金近1年的最大回撤及其同类排名信息是怎样的？,"('基金查询', '查询代码')('基金查询', '查询近期最大回撤')('基金查询', '查询近期最大回撤同类排名')","('基金查询', '查询代码')('基金查询', '查询近期最大回撤')('基金查询', '查询近期最大回撤同类排名')","[['汇添富添添鑫多元收益9个月持有期混合型证券投资基金A类']]['api_0的结果', '1年']['api_0的结果', '1年']","[['汇添富添添鑫多元收益9个月持有期混合型证券投资基金A类']]['api_0的结果', '1年']['api_0的结果', '1年']",3,3,"api_1,api_2","api_1,api_2"


### api_list

In [13]:
compare_api = compare_df.query('A_api_list!=B_api_list')[['query','A_api_list','B_api_list']]
print(len(compare_api))
disp(compare_api,cols_width=[300,300,300])

351


,query,A_api_list,B_api_list
1,我想了解汇丰晋信2016C的基金经理是谁，目前的基金规模以及申购费率是多少?,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')"
2,我现在要买1000份兴业聚惠A基金，我需要支付多少申购费？,"('基金查询', '查询代码')('基金查询', '查询申购费率')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询单位净值')('数值计算', '乘法计算')('数值计算', '乘法计算')"
7,我5000块要全投鹏华一带一路(LOF)A，扣除认购费用后真正可以投入的金额是多少,"('基金查询', '查询代码')('基金查询', '查询认购费率')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询认购费率')('数值计算', '乘法计算')('基金查询', '查询申购费率')('数值计算', '乘法计算')"
12,我想用500块买佛燃股票，舍弃当天初始成交的一半股票，我最后可以得到多少股股票？,"('股票查询', '查询代码')('股票查询', '查询开盘价')('数值计算', '除法计算')","('股票查询', '查询代码')('股票查询', '查询开盘价')('数值计算', '除法计算')('股票查询', '查询成交额')('数值计算', '除法计算')"
14,通过投资索菱的成交量和成交额，我想推测出本股的平均成交价格，并了解今天的主力资金流向。,"('股票查询', '查询代码')('股票查询', '查询成交量')('股票查询', '查询成交额')('数值计算', '除法计算')('股票查询', '查询主力资金流入')('股票查询', '查询主力资金流出')('数值计算', '减法计算')","('股票查询', '查询代码')('股票查询', '查询成交量')('股票查询', '查询成交额')('数值计算', '除法计算')('股票查询', '查询主力资金流入')"
15,我想知道我一个月前购买1000元的东兴连众C基金，扣除了销售服务费和认购费用后，我现在的总收益是多少,"('基金查询', '查询代码')('基金查询', '查询近期收益率')('基金查询', '查询销售服务费率')('基金查询', '查询认购费率')('数值计算', '乘法计算')('数值计算', '乘法计算')('数值计算', '乘法计算')('数值计算', '加法计算')('数值计算', '减法计算')","('基金查询', '查询代码')('基金查询', '查询销售服务费率')('基金查询', '查询认购费率')('数值计算', '乘法计算')('数值计算', '乘法计算')('数值计算', '减法计算')"
16,我想知道我昨天和今天一共买了200股川金诺，现在总共的市值是多少,"('股票查询', '查询代码')('股票查询', '查询当前价')","('股票查询', '查询代码')('股票查询', '查询收盘价')('股票查询', '查询收盘价')('数值计算', '乘法计算')('数值计算', '加法计算')"
19,我想知道如果我3个月前买入100股海联金汇，然后今天卖出，我需要支付的印花税是多少。,"('股票查询', '查询代码')('股票查询', '查询当前价')","('股票查询', '查询代码')('股票查询', '查询收盘价')('股票查询', '查询收盘价')('数值计算', '减法计算')"
21,我想知道如果我三个月前用5000块钱买了这个基金国投瑞银恒誉A，然后三个月后卖出，扣除申购费和赎回费我能赚到多少钱,"('基金查询', '查询代码')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('基金查询', '查询申购费率')('基金查询', '查询赎回费率')('数值计算', '加法计算')('数值计算', '乘法计算')('数值计算', '减法计算')","('基金查询', '查询代码')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('基金查询', '查询申购费率')('数值计算', '乘法计算')('数值计算', '减法计算')"
23,我要在两个月后用3000元申购银华美元债D，到时侯我需要付的申购费是多少,"('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')('数值计算', '加法计算')"


### params

In [14]:
compare_param = compare_df.query('A_api_list==B_api_list and A_api_params!=B_api_params')\
     [['query','A_api_list','B_api_list','A_api_params','B_api_params']]
print(len(compare_param))
disp(compare_param,cols_width=[200,300,300,300,300])

171


,query,A_api_list,B_api_list,A_api_params,B_api_params
3,南方新能源C的基金经理是谁，他管理的基金总规模有多大？,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')",[['南方新能源产业趋势混合型证券投资基金C类']]['api_1的结果'],[['南方中证新能源交易型开放式指数证券投资基金联接基金C类']]['api_1的结果']
6,我计划用10000块钱购买金鹰稳进配置C，我需要支付多少的申购费用,"('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","[['金鹰稳进配置六个月持有期混合型发起式基金中基金(FOF)C类']]['api_0的结果']['api_1的结果', '10000']","[['金鹰稳进配置六个月持有期混合型发起式基金中基金(FOF)C类']]['api_0的结果']['10000', 'api_1的结果']"
10,我想用5000块钱买鹏华宁华A基金，若我现在申购，需要支付多少费用？,"('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","[['鹏华宁华一年持有期混合型证券投资基金A类']]['api_0的结果']['api_1的结果', '5000']","[['鹏华宁华一年持有期混合型证券投资基金A类']]['api_0的结果']['5000', 'api_1的结果']"
13,请问世茂今天的开盘价和成交量是多少，以及主力资金的净流入情况如何？,"('股票查询', '查询代码')('股票查询', '查询开盘价')('股票查询', '查询成交量')('股票查询', '查询主力资金净流入')","('股票查询', '查询代码')('股票查询', '查询开盘价')('股票查询', '查询成交量')('股票查询', '查询主力资金净流入')","[['ST世茂']]['api_0的结果', '今日']['api_0的结果', '今日']","[['世茂股份']]['api_0的结果', '今日']['api_0的结果', '今日']"
27,我要用10000元买景顺长城量化平衡C基金，我需要付多少申购费？,"('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","('基金查询', '查询代码')('基金查询', '查询申购费率')('数值计算', '乘法计算')","[['景顺长城量化平衡灵活配置混合型证券投资基金C类']]['api_0的结果']['api_1的结果', '10000']","[['景顺长城量化平衡灵活配置混合型证券投资基金C类']]['api_0的结果']['10000', 'api_1的结果']"
38,如果我3个月前用2000块买了广发恒益A基金，然后又1个月前在市场上买了1000块的广发恒益A基金，现在整体盈利了多少？,"('基金查询', '查询代码')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('数值计算', '加法计算')","('基金查询', '查询代码')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('基金查询', '查询近期收益率')('数值计算', '乘法计算')('数值计算', '加法计算')","[['广发恒益一年持有期混合型证券投资基金A类']]['api_1的结果', '2000']['api_0的结果', '1个月']['api_3的结果', '1000']['api_2的结果', 'api_4的结果']","[['广发恒益A灵活配置混合型证券投资基金A类']]['api_1的结果', '2000']['api_0的结果', '1个月']['api_3的结果', '1000']['api_2的结果', 'api_4的结果']"
42,张江ETF现在的单位净值和近1个月的收益率同类排名情况如何？,"('基金查询', '查询代码')('基金查询', '查询单位净值')('基金查询', '查询近期收益率同类排名')","('基金查询', '查询代码')('基金查询', '查询单位净值')('基金查询', '查询近期收益率同类排名')","[['汇添富中证沪港深张江自主创新50交易型开放式指数证券投资基金']]['api_0的结果', '1个月']","[['张江高科']]['api_0的结果', '1个月']"
43,现在市面上有没有哪些股票，它们的每股净资产是负的，而且小于-4.49元呢？,"('条件选股', '查询每股净资产')","('条件选股', '查询每股净资产')","['小于', '-4.4868']","['等于', '-4.49', '今日']"
45,富国中证50C这个基金的风险等级和去年全年的超大盘收益率是怎样的？,"('基金查询', '查询代码')('基金查询', '查询风险等级')('基金查询', '查询近期超大盘收益率')","('基金查询', '查询代码')('基金查询', '查询风险等级')('基金查询', '查询近期超大盘收益率')","[['富国中证消费50交易型开放式指数证券投资基金联接基金C类']]['api_0的结果', '2022']","[['富国中证50交易型开放式指数证券投资基金']]['api_0的结果', '1年']"
46,昊华的流通市值和总市值分别是多少？,"('股票查询', '查询代码')('股票查询', '查询流通市值')('股票查询', '查询总市值')","('股票查询', '查询代码')('股票查询', '查询流通市值')('股票查询', '查询总市值')",[['昊华能源']]['api_0的结果'],[['昊华科技']]['api_0的结果']


### result

In [18]:
compare_result = compare_df.query('A_api_list==B_api_list and A_result!=B_result')\
     [['query','A_api_list','B_api_list','A_result','B_result']]
print(len(compare_result))
disp(compare_result,cols_width=[200,300,300,150,150])

15


,query,A_api_list,B_api_list,A_result,B_result
0,我想知道国金核心资产A的基金经理是谁，以及他的年化回报率和管理的总规模,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')('基金查询', '查询基金经理管理规模')","api_1,api_2,api_3","api_2,api_3"
89,上个月末比一个月前的股价各跌了0.99元的股票有哪些呢？,"('条件选股', '查询涨跌额')","('条件选股', '查询涨跌额')",api_0,"api_0,api_1"
121,请问现在财通智慧A的基金经理是谁？他有多少年的从业经验，并且管理的基金规模是多少?,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理从业年限')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理从业年限')('基金查询', '查询基金经理管理规模')","api_1,api_2,api_3","api_2,api_3"
209,我要知道豪迈今日的开盘价和收盘价的差值是多少，以及今日的主力资金流入和流出的比值,"('股票查询', '查询代码')('股票查询', '查询开盘价')('股票查询', '查询收盘价')('数值计算', '减法计算')('股票查询', '查询主力资金流入')('股票查询', '查询主力资金流出')('数值计算', '除法计算')","('股票查询', '查询代码')('股票查询', '查询开盘价')('股票查询', '查询收盘价')('数值计算', '减法计算')('股票查询', '查询主力资金流入')('股票查询', '查询主力资金流出')('数值计算', '除法计算')","api_3,api_6",api_6
242,我想知道中纺标今天的开盘价和收盘价之间的差价是多少，并且差价所占今天开盘价的比例是多少,"('股票查询', '查询代码')('股票查询', '查询开盘价')('股票查询', '查询收盘价')('数值计算', '减法计算')('数值计算', '除法计算')","('股票查询', '查询代码')('股票查询', '查询开盘价')('股票查询', '查询收盘价')('数值计算', '减法计算')('数值计算', '除法计算')","api_3,api_4",api_4
481,请问鹏扬中证科创创业50ETFA目前的单位净值是多少，同时我想了解一下其基金经理的年化回报率？,"('基金查询', '查询代码')('基金查询', '查询单位净值')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')","('基金查询', '查询代码')('基金查询', '查询单位净值')('基金查询', '查询基金经理')('基金查询', '查询基金经理年化回报率')","api_1,api_3","api_1,api_2,api_3"
504,通号的股票代码是什么，以及它昨日的收盘价和今日的开盘价分别是多少？,"('股票查询', '查询代码')('股票查询', '查询收盘价')('股票查询', '查询开盘价')","('股票查询', '查询代码')('股票查询', '查询收盘价')('股票查询', '查询开盘价')","api_0,api_1,api_2","api_1,api_2"
561,请问华夏网购精选C基金是由哪位基金经理管理的，同时给出该基金经理的从业年限和管理的总规模。,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理从业年限')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理从业年限')('基金查询', '查询基金经理管理规模')","api_1,api_2,api_3","api_2,api_3"
576,中银远见成长C的基金经理目前管理的总规模和该基金自成立以来的最大回撤是多少？,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')('基金查询', '查询近期最大回撤')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理管理规模')('基金查询', '查询近期最大回撤')","api_2,api_3","api_1,api_2,api_3"
629,想了解易方达中证800ETFC的基金经理任职期限和他管理的基金规模,"('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理任职期限')('基金查询', '查询基金经理管理规模')","('基金查询', '查询代码')('基金查询', '查询基金经理')('基金查询', '查询基金经理任职期限')('基金查询', '查询基金经理管理规模')","api_2,api_3","api_1,api_2,api_3"
